# SLAM tutorial

SLAM and factor graphs tutorial prepared for the ORIentate Seminars

Author: Matias Mattamala (matias@robots.ox.ac.uk, [mmattamala@github](https://github.com/mmattamala))

Date: 23/11/2023


> ⚠️ **Warning**: Before proceeding, make sure you installed all the required dependencies in the [`requirements.txt`](../requirements.txt) file!


## Preliminaries

In [ ]:
%pip install -e ../

## Inspecting a SLAM graph

In [ ]:
import slam_tutorial

In [ ]:
from slam_tutorial.io import load_ground_truth
df = load_ground_truth(slam_tutorial.ASSETS_DIR + "/graphs/ground_truth.csv")

In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd

viz_clouds = []
x_tm1 = y_tm1 = z_tm1 = None
for s, ns, x_t, y_t, z_t, qx_t, qy_t, qz_t, qw_t in zip(df['#sec'], df['nsec'], df['x'], df['y'], df['z'], df['qx'], df['qy'], df['qz'], df['qw']):
    pose = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=np.array([x_t, y_t, z_t]))
    viz_clouds.append(pose)

In [ ]:
# o3d.visualization.draw_geometries(
#             viz_clouds,
#             window_name="ground_truth",
# )

In [ ]:
from slam_tutorial.io import load_pose_graph
graph = load_pose_graph(slam_tutorial.ASSETS_DIR + "/graphs/slam_pose_graph.slam")

In [ ]:
# graph.show()

In [ ]:
import gtsam
from gtsam import Pose3

# 1. Create a factor graph container and add factors to it
factor_graph = gtsam.NonlinearFactorGraph()

# Add prior
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(0.001 * np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1]))
factor_graph.add(gtsam.PriorFactorPose3(0, gtsam.Pose3(graph.get_node_pose(0).flatten()), PRIOR_NOISE))

# Add odometry
for e in graph.edges:
    if e["type"] == "odometry":
        ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Information(e["info"])
        factor_graph.add(gtsam.BetweenFactorPose3(e["parent_id"], e["child_id"], gtsam.Pose3(e["pose"].flatten()), ODOMETRY_NOISE))

# Add initial values
initial_estimate = gtsam.Values()
for id, node in graph.nodes.items():
    initial_estimate.insert(id, gtsam.Pose3(graph.get_node_pose(id).flatten()))

# Setup optimizer
# parameters = gtsam.GaussNewtonParams()
# optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)
# optimizer.optimize()

parameters = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(factor_graph, initial_estimate, parameters)

# Manually run optimizer
for i in range(10):
    optimizer.iterate()
    sol = optimizer.values()
    for n,_ in enumerate(graph.nodes):
        graph.set_node_pose(n, sol.atPose3(n).matrix())
    # print(optimizer.values().atPose3(0))

# Compute marginals to show covariances
marginals = gtsam.Marginals(factor_graph, optimizer.values())

In [ ]:
# factor_graph.saveGraph("test.dot", initial_estimate)
# from graphviz import Source
# s = Source.from_file("test.dot")
# s.view()

In [ ]:
# vis = o3d.visualization.Visualizer()
# vis.create_window()

# # geometry is the point cloud used in your animaiton
# geometry = o3d.geometry.PointCloud()
# vis.add_geometry(geometry)

# for i in range(icp_iteration):
#     # now modify the points of your geometry
#     # you can use whatever method suits you best, this is just an example
#     geometry.points = pcd_list[i].points
#     vis.update_geometry(geometry)
#     vis.poll_events()
#     vis.update_renderer()